In [47]:
from google.colab import files  # из библиотеки google.colab импортируем класс files
import pandas as pd             # для вывода данных
#import seaborn as sns           # для построения графиков воспользуемся новой для нас библиотекой seaborn
from sklearn.preprocessing import StandardScaler                          # импортируем класс StandardScaler для нормализации данных

In [48]:
uploaded = files.upload()       # загружаем файл с данными с компьютера

In [49]:
train = pd.read_csv('/content/milk_dataset_1.csv')      # применим функцию read_csv() и посмотрим на первые три записи файла train.csv
train.head(3)

,Proteins (g),Fats (g),Carbohydrates (g),Calories,Days until the end of the shelf life,Weight (kg),Height (cm),Age,Physique,Individual intolerance,Usefulness of the product for humans
0,2.9,2.5,4.7,53.0,10,70,175,25,2,1,0
1,2.9,2.5,4.7,53.0,9,58,163,32,1,1,0
2,2.9,2.5,4.7,53.0,8,85,187,40,3,1,0


In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Proteins (g)                          184 non-null    float64
 1   Fats (g)                              184 non-null    float64
 2   Carbohydrates (g)                     184 non-null    float64
 3   Calories                              184 non-null    float64
 4   Days until the end of the shelf life  184 non-null    int64  
 5   Weight (kg)                           184 non-null    int64  
 6   Height (cm)                           184 non-null    int64  
 7   Age                                   184 non-null    int64  
 8   Physique                              184 non-null    int64  
 9   Individual intolerance                184 non-null    int64  
 10  Usefulness of the product for humans  184 non-null    int64  
dtypes: float64(4), int6

Поработаем с категориальными переменными (categorical variable).
  
Т.к. категории телосложений выражены числами 1(худощавое), 2(среднее), 3(спортивное), 4(полное), то модель воспримет их в качестве количественных переменных, а не категориальных, поэтопу применим one-hot encoding. В библиотеке Pandas есть метод .get_dummies(), который как раз и выполнит необходимые преобразования.

In [51]:
physique = pd.get_dummies(train['Physique'], drop_first = True)       # разбиваем телосложение на категории "1", "2", "3" и "4", затем отбрасываем столбец "1", т.к. он избыточен

In [52]:
train = pd.concat([train, physique], axis = 1)              # проводим конкатенацию нашего основного набора данных с полученным ранее набором категорий телосложения
train

,Proteins (g),Fats (g),Carbohydrates (g),Calories,Days until the end of the shelf life,Weight (kg),Height (cm),Age,Physique,Individual intolerance,Usefulness of the product for humans,2,3,4
0,2.9,2.5,4.7,53.0,10,70,175,25,2,1,0,1,0,0
1,2.9,2.5,4.7,53.0,9,58,163,32,1,1,0,0,0,0
2,2.9,2.5,4.7,53.0,8,85,187,40,3,1,0,0,1,0
3,2.9,2.5,4.7,53.0,7,62,158,19,3,1,0,0,1,0
4,2.9,2.5,4.7,53.0,6,90,195,45,2,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2.0,0.1,4.8,31.0,7,62,158,25,4,0,1,0,0,1
180,2.0,0.1,4.8,31.0,7,62,158,40,4,0,1,0,0,1
181,2.0,0.1,4.8,31.0,7,62,158,60,4,0,1,0,0,1
182,2.0,0.1,4.8,31.0,7,62,158,85,4,0,1,0,0,1


In [53]:
train.drop(['Physique'], axis = 1, inplace = True)      # избавляемся от уже ненужного столбца 'Physique'
train.head(3)                                           # показываем получившийся набор данных

,Proteins (g),Fats (g),Carbohydrates (g),Calories,Days until the end of the shelf life,Weight (kg),Height (cm),Age,Individual intolerance,Usefulness of the product for humans,2,3,4
0,2.9,2.5,4.7,53.0,10,70,175,25,1,0,1,0,0
1,2.9,2.5,4.7,53.0,9,58,163,32,1,0,0,0,0
2,2.9,2.5,4.7,53.0,8,85,187,40,1,0,0,1,0


Оставшиеся переменные представляют собой либо количественные, либо категориальные признаки, выражены через 0 и 1.

Для избежания ситуации, в которой модель неоправданно придает большее значение признаку с большей числовой характеристикой, получая соответственно ошибочный результат, проведем нормализацию данных и приведем количественные переменные к одному масштабу.

In [54]:
scaler = StandardScaler()                                                 # создадим объект класса StandardScaler
cols_to_scale = ['Proteins (g)', 'Fats (g)', 'Carbohydrates (g)',
                 'Calories', 'Days until the end of the shelf life',
                 'Weight (kg)', 'Height (cm)', 'Age']                     # выберем те столбцы, которые мы хотим масштабировать
scaler.fit(train[cols_to_scale])                                          # рассчитаем среднее арифметическое и СКО для масштабирования данных
train[cols_to_scale] = scaler.transform(train[cols_to_scale])             # нормализуем данные

train                                                                     # посмотрим на результат

,Proteins (g),Fats (g),Carbohydrates (g),Calories,Days until the end of the shelf life,Weight (kg),Height (cm),Age,Individual intolerance,Usefulness of the product for humans,2,3,4
0,0.532014,0.120858,-1.455556,0.106348,1.876773,0.141854,0.381744,-0.547051,1,0,1,0,0
1,0.532014,0.120858,-1.455556,0.106348,1.548828,-0.611065,-0.366162,-0.232637,1,0,0,0,0
2,0.532014,0.120858,-1.455556,0.106348,1.220883,1.083003,1.129650,0.126693,1,0,0,1,0
3,0.532014,0.120858,-1.455556,0.106348,0.892937,-0.360092,-0.677790,-0.816549,1,0,0,1,0
4,0.532014,0.120858,-1.455556,0.106348,0.564992,1.396719,1.628254,0.351275,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.318858,-1.200201,0.687023,-1.194248,0.892937,-0.360092,-0.677790,-0.547051,0,1,0,0,1
180,-1.318858,-1.200201,0.687023,-1.194248,0.892937,-0.360092,-0.677790,0.126693,0,1,0,0,1
181,-1.318858,-1.200201,0.687023,-1.194248,0.892937,-0.360092,-0.677790,1.025020,0,1,0,0,1
182,-1.318858,-1.200201,0.687023,-1.194248,0.892937,-0.360092,-0.677790,2.147928,0,1,0,0,1


In [55]:
train.columns = train.columns.map(str)               # приведем все названия колонок к строковому формату

In [56]:
X_all = train.drop('Usefulness of the product for humans', axis = 1)         # поместим в X_all все кроме столбца 'Usefulness of the product for humans', т.е. нашего target
y_all = train['Usefulness of the product for humans']                        # столбец 'Usefulness of the product for humans' станет нашей целевой переменной

In [57]:
X_all.head(3)

,Proteins (g),Fats (g),Carbohydrates (g),Calories,Days until the end of the shelf life,Weight (kg),Height (cm),Age,Individual intolerance,2,3,4
0,0.532014,0.120858,-1.455556,0.106348,1.876773,0.141854,0.381744,-0.547051,1,1,0,0
1,0.532014,0.120858,-1.455556,0.106348,1.548828,-0.611065,-0.366162,-0.232637,1,0,0,0
2,0.532014,0.120858,-1.455556,0.106348,1.220883,1.083003,1.129650,0.126693,1,0,1,0


In [58]:
X=X_all.values        # для большего удобства приведем наши данные к типу numpy.ndarray
y=y_all.values
X

array([[ 0.53201396,  0.12085776, -1.45555627, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.53201396,  0.12085776, -1.45555627, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.53201396,  0.12085776, -1.45555627, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.31885814, -1.20020133,  0.68702256, ...,  0.        ,
         0.        ,  1.        ],
       [-1.31885814, -1.20020133,  0.68702256, ...,  0.        ,
         0.        ,  1.        ],
       [-1.31885814, -1.20020133,  0.68702256, ...,  0.        ,
         0.        ,  1.        ]])